In [1]:
import sys; sys.path.append("..")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact

from config import *

sns.set_style("whitegrid")

In [2]:
# settings
# skip obs with n_days till next announcement being more than this
MAX_DAYS_TO_ANNOUNCEMENT = 10

# data
For each day and currency, we observe the the FX spot return (vs USD), the number of days until the next 'own' and FOMC policy meeting, whether they are scheduled or not, and the direction of the rate change that they culminated in. For instance, on 2001-01-17 there were 6 days until the next policy meeting at the Bank of Canada, and 14 days until the next FOMC meeting, both scheduled and to result in a rate cut.

In [3]:
data = pd.read_csv("../data/processed/data.csv")

data.head(1).style.format()

,currency,date,date_announcement_own,n_days_own,fx_return,direction_own,is_scheduled_own,date_announcement_fomc,n_days_fomc,direction_fomc,is_scheduled_fomc
0,cad,2000-01-03 16:00:00-05:00,2000-12-05 10:00:00-05:00,-241,nan,no_change,True,2000-02-02 14:00:00-05:00,-22,hike,True


# analysis

Let's take a look at the historical dynamics of spot exchange rates ahead of monetary policy meetings, irrespective of the policy decision:

In [22]:
@interact
def plot_pre_announcement_spot_returns(
        which_bank=["own", "fomc"],
        years=widgets.IntRangeSlider(
            value=[2000, 2023],
            min=2000,
            max=2023,
            step=1,
            description='years',
            continuous_update=False
        )
    ):

    q_bank = f"-{MAX_DAYS_TO_ANNOUNCEMENT} <= n_days_{which_bank}"
    q_years = f"'{years[0]}' <= date_announcement_{which_bank} <= '{years[1]}'"

    _, ax = plt.subplots()
    sns.barplot(
        data=data.query(f"is_scheduled_{which_bank}")\
                 .query(q_bank)\
                 .query(q_years),
        x=f"n_days_{which_bank}", 
        y="fx_return", 
        errorbar=None,
        ax=ax
    )
    # align fomc that do no have 0-day
    ax.set_xlim((-0.5, -0.5+MAX_DAYS_TO_ANNOUNCEMENT+1))
    ax.set_xlabel("days to meeting")
    ax.set_ylabel("spot return")
    ax.grid(axis="x")
    plt.plot()

interactive(children=(Dropdown(description='which_bank', options=('own', 'fomc'), value='own'), IntRangeSlider…

Now, let's redo figure 1 with a breakdown into rate cuts, hikes and no-changes.

In [24]:
@interact
def plot_pre_announcement_spot_returns_by_decision(
        which_bank=["own", "fomc"],
        years=widgets.IntRangeSlider(
            value=[2000, 2023],
            min=2000,
            max=2023,
            step=1,
            description='years',
            continuous_update=False
        )
    ):
    q_bank = f"-{MAX_DAYS_TO_ANNOUNCEMENT} <= n_days_{which_bank}"
    q_years = f"'{years[0]}' <= date_announcement_{which_bank} <= '{years[1]}'"

    _, ax = plt.subplots()
    sns.barplot(
        data=data.query(f"is_scheduled_{which_bank}")
                 .query(q_bank)\
                 .query(q_years),
        x=f"n_days_{which_bank}", 
        errorbar=None,
        y="fx_return",
        ax=ax,
        hue=f"direction_{which_bank}", 
        hue_order=["cut", "hike", "no_change"]
    )
    # align fomc that do no have 0-day
    ax.set_xlim((-0.5, -0.5+MAX_DAYS_TO_ANNOUNCEMENT+1))
    ax.set_xlabel("days to meeting")
    ax.set_ylabel("spot return")
    ax.grid(axis="x")

interactive(children=(Dropdown(description='which_bank', options=('own', 'fomc'), value='own'), IntRangeSlider…